In [89]:
import requests
from bs4 import BeautifulSoup as bs

In [189]:
r = requests.get("https://www.kufar.by/l?ot=1&query=кольцо&rgn=all&sort=lst.d&utm_queryOrigin=Manually_typed")

In [190]:
info = bs(r.text, "html.parser")

In [191]:
ring_names = info.find_all('a', class_='styles_wrapper__5FoK7')

In [192]:
import threading
from concurrent.futures import ThreadPoolExecutor

In [193]:
def get_by(href, array):
    array.append(requests.get((href)))

In [200]:
all_requests = []
with ThreadPoolExecutor(max_workers=4) as executor:
        for elem in ring_names:
            executor.submit(get_by, elem["href"], all_requests)
print(len(all_requests))

54


In [201]:
request_html = [bs(elem.text, "html.parser") for elem in all_requests]

In [212]:
all_descriptions = [elem.find_all('div', itemprop = "description") for elem in request_html]
all_prices = [elem.find_all(['span','div'], class_ = ["styles_discountPrice__WuQiu", "styles_price--main__KHbAp"])[0].text for elem in request_html]
print(all_prices)

['500 р.', '19.99 р.', 'Договорная', '35 р.', '290 р.', '450 р.', '15 р.', '12 р.', '19 р.', '19.99 р.', '29.99 р.', '130 р.', '1 100 р.', '57 р.', 'Договорная', '30 р.', '20 р.', '600 р.', '30.54 р.', '600 р.', '30 р.', '20 р.', '45 р.', '800 р.', '20 р.', '2 р.', '45 р.', '15 р.', '320 р.', '20 р.', 'Договорная', '199 р.', '33 р.', '20 р.', '30 р.', '100 р.', '18 р.', '70 р.', 'Договорная', '25 р.', '23.50 р.', '45 р.', '20 р.', '300 р.', '12 р.', '350 р.', '19 р.', '10 р.', '19.99 р.', '29.99 р.', '30.54 р.', '57 р.', '33 р.', '68 р.']


In [213]:
def check_if_in_or_not_in_string(words: list[str], line: str, in_or_not:bool) -> bool:
    if in_or_not:
        for word in words:
            if word not in line:
                return False
        return True
    else:
        for word in words:
            if word in line:
                return False
        return True

In [214]:
def find_size(s: str, index: int) -> str:
    tmp = s[index + 7: index + 11]
    k = "".join([elem for elem in tmp if elem.isnumeric()])
    if k != "":
        return k
    else:
        tmp = s[index - 4: index]
        return "".join([elem for elem in tmp if elem.isnumeric()])

In [216]:
good_indexes = []
size = {}
in_words, not_in_words = ["золот"], ["позоло", "сереб", "бронз"]
for num, elem in enumerate(all_descriptions):
    my_text = elem[0].text.lower()
    if check_if_in_or_not_in_string(in_words, my_text, True) and check_if_in_or_not_in_string(not_in_words, my_text, False):
        good_indexes.append(num)
        if (k := my_text.find("размер")) != -1:
            size[num] = find_size(my_text, k)
print(good_indexes)
print(size)

[0, 4, 8, 12, 19, 23, 24, 34, 43, 46]
{4: '175', 12: '175', 19: '175', 23: '', 24: '19', 43: '15'}


In [217]:
with open("test.txt", "w") as file:
    for index in good_indexes:
        if index in size:
            file.write(all_prices[index] + " " +  size[index] + " " + ring_names[index]["href"] + '\n')
        else:
            file.write(all_prices[index] + " " + ring_names[index]["href"] + '\n')